In [1]:
import geopandas as gpd
import requests
from zipfile import ZipFile
import io
import pandas as pd

In [2]:
local_path = "./dataset/"

### Loading label data AUS

In [3]:
df = pd.read_csv(f"{local_path}2020-2023_filtered_labels_bushfire.csv")
df

,Unnamed: 0,...1,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,VerDate,geometry,Class_Label,SA2_CODE21,Year
0,1,244,River Flats HR,HR20062388730,2020-21 Prescribed Burn,2020-08-06,2020-08-14,3197.550793,77842.548467,12/10/23,"POLYGON ((152.31536609727 -30.15763724021, 152...",Prescribed Burn,110011187,2020
1,2,246,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,89.573878,10000.249608,12/10/23,"POLYGON ((152.31033334635 -30.56818758203172, ...",Prescribed Burn,110011187,2021
2,3,247,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,99.588438,9685.554813,12/10/23,"POLYGON ((152.328781873 -30.554755261826518, 1...",Prescribed Burn,110011187,2021
3,4,248,Long Point HR,BA-2021-00261,2022-23 Prescribed Burn,2023-06-17,2023-06-22,1818.784105,42622.260319,12/10/23,"POLYGON ((151.98061160649 -30.77536483822499, ...",Prescribed Burn,110011187,2023
4,5,249,Long Point Stage 2 HR,BA20230706000544,2023-24 Prescribed Burn,2023-08-11,2023-08-19,327.969953,10617.367261,12/10/23,"POLYGON ((151.99066226219 -30.733447633369337,...",Prescribed Burn,110011187,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,1498,13405,Willoughbys Connection HR,HR20080389083,2020-21 Prescribed Burn,2021-04-19,2021-05-27,19.829443,3453.555104,12/10/23,MULTIPOLYGON (((144.98427525145 -35.5323346707...,Prescribed Burn,109031182,2021
1498,1499,13406,Joggers Trail HR,HR21020190786,2020-21 Prescribed Burn,2021-04-18,2021-04-27,86.028493,3937.302123,12/10/23,"POLYGON ((144.98422473385 -35.54784276085, 144...",Prescribed Burn,109031182,2021
1499,1500,13407,Echidna Trail Murray valley RP Deniliquin,22011241487,2021-22 Wildfire,2022-01-11,2022-01-13,0.001593,16.069846,12/10/23,"POLYGON ((144.9794526408 -35.5440399793488, 14...",Wildfire,109031182,2022
1500,1501,13408,Bullawa Creek West HR,BA-2021-00075,2022-23 Prescribed Burn,2023-04-21,2023-04-24,17.198572,1756.403095,12/10/23,"POLYGON ((149.88990682192 -30.337989949187058,...",Prescribed Burn,110031197,2023


In [4]:
df = df.drop(["Unnamed: 0", "...1"], axis=1)
df

,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,VerDate,geometry,Class_Label,SA2_CODE21,Year
0,River Flats HR,HR20062388730,2020-21 Prescribed Burn,2020-08-06,2020-08-14,3197.550793,77842.548467,12/10/23,"POLYGON ((152.31536609727 -30.15763724021, 152...",Prescribed Burn,110011187,2020
1,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,89.573878,10000.249608,12/10/23,"POLYGON ((152.31033334635 -30.56818758203172, ...",Prescribed Burn,110011187,2021
2,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,99.588438,9685.554813,12/10/23,"POLYGON ((152.328781873 -30.554755261826518, 1...",Prescribed Burn,110011187,2021
3,Long Point HR,BA-2021-00261,2022-23 Prescribed Burn,2023-06-17,2023-06-22,1818.784105,42622.260319,12/10/23,"POLYGON ((151.98061160649 -30.77536483822499, ...",Prescribed Burn,110011187,2023
4,Long Point Stage 2 HR,BA20230706000544,2023-24 Prescribed Burn,2023-08-11,2023-08-19,327.969953,10617.367261,12/10/23,"POLYGON ((151.99066226219 -30.733447633369337,...",Prescribed Burn,110011187,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
1497,Willoughbys Connection HR,HR20080389083,2020-21 Prescribed Burn,2021-04-19,2021-05-27,19.829443,3453.555104,12/10/23,MULTIPOLYGON (((144.98427525145 -35.5323346707...,Prescribed Burn,109031182,2021
1498,Joggers Trail HR,HR21020190786,2020-21 Prescribed Burn,2021-04-18,2021-04-27,86.028493,3937.302123,12/10/23,"POLYGON ((144.98422473385 -35.54784276085, 144...",Prescribed Burn,109031182,2021
1499,Echidna Trail Murray valley RP Deniliquin,22011241487,2021-22 Wildfire,2022-01-11,2022-01-13,0.001593,16.069846,12/10/23,"POLYGON ((144.9794526408 -35.5440399793488, 14...",Wildfire,109031182,2022
1500,Bullawa Creek West HR,BA-2021-00075,2022-23 Prescribed Burn,2023-04-21,2023-04-24,17.198572,1756.403095,12/10/23,"POLYGON ((149.88990682192 -30.337989949187058,...",Prescribed Burn,110031197,2023


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   FireName     1502 non-null   object 
 1   FireNo       1502 non-null   object 
 2   Label        1502 non-null   object 
 3   StartDate    1502 non-null   object 
 4   EndDate      1502 non-null   object 
 5   AreaHa       1502 non-null   float64
 6   PerimeterM   1502 non-null   float64
 7   VerDate      1502 non-null   object 
 8   geometry     1502 non-null   object 
 9   Class_Label  1502 non-null   object 
 10  SA2_CODE21   1502 non-null   int64  
 11  Year         1502 non-null   int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 140.9+ KB


In [6]:
from datetime import datetime

In [7]:
df['StartDate'] = df['StartDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
df.head()

,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,VerDate,geometry,Class_Label,SA2_CODE21,Year
0,River Flats HR,HR20062388730,2020-21 Prescribed Burn,2020-08-06,2020-08-14,3197.550793,77842.548467,12/10/23,"POLYGON ((152.31536609727 -30.15763724021, 152...",Prescribed Burn,110011187,2020
1,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,89.573878,10000.249608,12/10/23,"POLYGON ((152.31033334635 -30.56818758203172, ...",Prescribed Burn,110011187,2021
2,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,99.588438,9685.554813,12/10/23,"POLYGON ((152.328781873 -30.554755261826518, 1...",Prescribed Burn,110011187,2021
3,Long Point HR,BA-2021-00261,2022-23 Prescribed Burn,2023-06-17,2023-06-22,1818.784105,42622.260319,12/10/23,"POLYGON ((151.98061160649 -30.77536483822499, ...",Prescribed Burn,110011187,2023
4,Long Point Stage 2 HR,BA20230706000544,2023-24 Prescribed Burn,2023-08-11,2023-08-19,327.969953,10617.367261,12/10/23,"POLYGON ((151.99066226219 -30.733447633369337,...",Prescribed Burn,110011187,2023


In [8]:
df['EndDate'] = df['EndDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
df.head()

,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,VerDate,geometry,Class_Label,SA2_CODE21,Year
0,River Flats HR,HR20062388730,2020-21 Prescribed Burn,2020-08-06,2020-08-14,3197.550793,77842.548467,12/10/23,"POLYGON ((152.31536609727 -30.15763724021, 152...",Prescribed Burn,110011187,2020
1,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,89.573878,10000.249608,12/10/23,"POLYGON ((152.31033334635 -30.56818758203172, ...",Prescribed Burn,110011187,2021
2,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,99.588438,9685.554813,12/10/23,"POLYGON ((152.328781873 -30.554755261826518, 1...",Prescribed Burn,110011187,2021
3,Long Point HR,BA-2021-00261,2022-23 Prescribed Burn,2023-06-17,2023-06-22,1818.784105,42622.260319,12/10/23,"POLYGON ((151.98061160649 -30.77536483822499, ...",Prescribed Burn,110011187,2023
4,Long Point Stage 2 HR,BA20230706000544,2023-24 Prescribed Burn,2023-08-11,2023-08-19,327.969953,10617.367261,12/10/23,"POLYGON ((151.99066226219 -30.733447633369337,...",Prescribed Burn,110011187,2023


In [9]:
import shapely.wkt

In [10]:
df['geometry'] = df.geometry.apply(lambda x: shapely.wkt.loads(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   FireName     1502 non-null   object 
 1   FireNo       1502 non-null   object 
 2   Label        1502 non-null   object 
 3   StartDate    1502 non-null   object 
 4   EndDate      1502 non-null   object 
 5   AreaHa       1502 non-null   float64
 6   PerimeterM   1502 non-null   float64
 7   VerDate      1502 non-null   object 
 8   geometry     1502 non-null   object 
 9   Class_Label  1502 non-null   object 
 10  SA2_CODE21   1502 non-null   int64  
 11  Year         1502 non-null   int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 140.9+ KB


In [11]:
df.head()

,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,VerDate,geometry,Class_Label,SA2_CODE21,Year
0,River Flats HR,HR20062388730,2020-21 Prescribed Burn,2020-08-06,2020-08-14,3197.550793,77842.548467,12/10/23,"POLYGON ((152.31536609727 -30.15763724021, 152...",Prescribed Burn,110011187,2020
1,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,89.573878,10000.249608,12/10/23,"POLYGON ((152.31033334635 -30.56818758203172, ...",Prescribed Burn,110011187,2021
2,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,99.588438,9685.554813,12/10/23,"POLYGON ((152.328781873 -30.554755261826518, 1...",Prescribed Burn,110011187,2021
3,Long Point HR,BA-2021-00261,2022-23 Prescribed Burn,2023-06-17,2023-06-22,1818.784105,42622.260319,12/10/23,"POLYGON ((151.98061160649 -30.77536483822499, ...",Prescribed Burn,110011187,2023
4,Long Point Stage 2 HR,BA20230706000544,2023-24 Prescribed Burn,2023-08-11,2023-08-19,327.969953,10617.367261,12/10/23,"POLYGON ((151.99066226219 -30.733447633369337,...",Prescribed Burn,110011187,2023


In [12]:
gpd_2 = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=[x for x in df.geometry])
gpd_2.head()

,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,VerDate,geometry,Class_Label,SA2_CODE21,Year
0,River Flats HR,HR20062388730,2020-21 Prescribed Burn,2020-08-06,2020-08-14,3197.550793,77842.548467,12/10/23,"POLYGON ((152.31537 -30.15764, 152.31605 -30.1...",Prescribed Burn,110011187,2020
1,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,89.573878,10000.249608,12/10/23,"POLYGON ((152.31033 -30.56819, 152.31110 -30.5...",Prescribed Burn,110011187,2021
2,Diamond Flat HR,BA-2020-01498,2021-22 Prescribed Burn,2021-07-28,2021-08-01,99.588438,9685.554813,12/10/23,"POLYGON ((152.32878 -30.55476, 152.32881 -30.5...",Prescribed Burn,110011187,2021
3,Long Point HR,BA-2021-00261,2022-23 Prescribed Burn,2023-06-17,2023-06-22,1818.784105,42622.260319,12/10/23,"POLYGON ((151.98061 -30.77536, 151.98063 -30.7...",Prescribed Burn,110011187,2023
4,Long Point Stage 2 HR,BA20230706000544,2023-24 Prescribed Burn,2023-08-11,2023-08-19,327.969953,10617.367261,12/10/23,"POLYGON ((151.99066 -30.73345, 151.99085 -30.7...",Prescribed Burn,110011187,2023


### Merging actual data

In [13]:
df_to_merged = pd.read_csv(f"{local_path}data_Australia_filter.csv")
df_to_merged

,Unnamed: 0,...1,stn_id,stn_call,city,state,country,date,lat,lon,...,at,wc,wspd_3,wspd_6,wspd_9,wspd_12,wspd_15,wspd_18,wspd_21,wspd_24
0,1,1.0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,...,40.2,25.1,6,2,9,7,6,3,0,5
1,2,2.0,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,...,40.9,29.0,6,7,12,9,7,7,10,12
2,3,3.0,94103,-99999,BROWSE_ISL_AWS(AUT),-99999,AUSTRALIA,20200101,-14.1200,123.5475,...,-99999.0,27.2,19,7,4,12,13,11,11,10
3,4,4.0,94105,-99999,NOONAMAH_AIRSTRIP,-99999,AUSTRALIA,20200101,-12.7000,131.0475,...,40.5,23.6,2,6,3,0,1,3,3,2
4,5,5.0,94106,-99999,NGUKURR_AIRPORT,-99999,AUSTRALIA,20200101,-14.7236,134.7456,...,40.0,26.9,5,8,12,1,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839333,839334,179147.0,95985,-99999,SWAN_ISLAND,-99999,AUSTRALIA,20231103,-40.7292,148.1250,...,16.9,7.1,19,21,22,20,11,11,10,10
839334,839335,179148.0,95986,-99999,TASMAN_ISLAND,-99999,AUSTRALIA,20231103,-43.2397,148.0025,...,10.3,3.4,17,13,13,12,8,8,5,11
839335,839336,179149.0,95988,-99999,MARIA_ISLAND_(POINT_LESUEUR),-99999,AUSTRALIA,20231103,-42.6619,148.0181,...,12.6,5.2,22,25,16,15,12,10,6,15
839336,839337,179150.0,95989,-99999,BICHENO_(COUNCIL_DEPOT),-99999,AUSTRALIA,20231103,-41.8739,148.3036,...,12.2,11.1,-99999,-99999,-99999,-99999,-99999,-99999,8,-99999


In [14]:
df_to_merged = df_to_merged.drop(["Unnamed: 0", "...1"], axis=1)

In [15]:
df_to_merged.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,at,wc,wspd_3,wspd_6,wspd_9,wspd_12,wspd_15,wspd_18,wspd_21,wspd_24
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,40.2,25.1,6,2,9,7,6,3,0,5
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,40.9,29.0,6,7,12,9,7,7,10,12
2,94103,-99999,BROWSE_ISL_AWS(AUT),-99999,AUSTRALIA,20200101,-14.1200,123.5475,5,32.4,...,-99999.0,27.2,19,7,4,12,13,11,11,10
3,94105,-99999,NOONAMAH_AIRSTRIP,-99999,AUSTRALIA,20200101,-12.7000,131.0475,18,32.8,...,40.5,23.6,2,6,3,0,1,3,3,2
4,94106,-99999,NGUKURR_AIRPORT,-99999,AUSTRALIA,20200101,-14.7236,134.7456,16,37.7,...,40.0,26.9,5,8,12,1,0,0,1,3


In [16]:
df_to_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839338 entries, 0 to 839337
Data columns (total 35 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   stn_id     839338 non-null  int64  
 1   stn_call   839338 non-null  object 
 2   city       839338 non-null  object 
 3   state      839338 non-null  int64  
 4   country    839338 non-null  object 
 5   date       839338 non-null  int64  
 6   lat        839338 non-null  float64
 7   lon        839338 non-null  float64
 8   elev       839338 non-null  int64  
 9   tmax       839338 non-null  float64
 10  tmin       839338 non-null  float64
 11  report_p   839338 non-null  float64
 12  final_p    839338 non-null  float64
 13  p_flag     839338 non-null  int64  
 14  num_6hr_p  839338 non-null  int64  
 15  wxchars    839338 non-null  object 
 16  trace      839338 non-null  int64  
 17  vp         839338 non-null  float64
 18  vp_def     839338 non-null  float64
 19  slp_6      839338 non-n

In [17]:
df_to_merged['date'] = df_to_merged['date'].apply(lambda x: f"{x}")
df_to_merged.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,at,wc,wspd_3,wspd_6,wspd_9,wspd_12,wspd_15,wspd_18,wspd_21,wspd_24
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,40.2,25.1,6,2,9,7,6,3,0,5
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,40.9,29.0,6,7,12,9,7,7,10,12
2,94103,-99999,BROWSE_ISL_AWS(AUT),-99999,AUSTRALIA,20200101,-14.1200,123.5475,5,32.4,...,-99999.0,27.2,19,7,4,12,13,11,11,10
3,94105,-99999,NOONAMAH_AIRSTRIP,-99999,AUSTRALIA,20200101,-12.7000,131.0475,18,32.8,...,40.5,23.6,2,6,3,0,1,3,3,2
4,94106,-99999,NGUKURR_AIRPORT,-99999,AUSTRALIA,20200101,-14.7236,134.7456,16,37.7,...,40.0,26.9,5,8,12,1,0,0,1,3


In [18]:
df_to_merged['date_new'] = df_to_merged['date'].apply(lambda x: datetime.strptime(x, "%Y%m%d").date())

In [19]:
df_to_merged

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,wc,wspd_3,wspd_6,wspd_9,wspd_12,wspd_15,wspd_18,wspd_21,wspd_24,date_new
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,25.1,6,2,9,7,6,3,0,5,2020-01-01
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,29.0,6,7,12,9,7,7,10,12,2020-01-01
2,94103,-99999,BROWSE_ISL_AWS(AUT),-99999,AUSTRALIA,20200101,-14.1200,123.5475,5,32.4,...,27.2,19,7,4,12,13,11,11,10,2020-01-01
3,94105,-99999,NOONAMAH_AIRSTRIP,-99999,AUSTRALIA,20200101,-12.7000,131.0475,18,32.8,...,23.6,2,6,3,0,1,3,3,2,2020-01-01
4,94106,-99999,NGUKURR_AIRPORT,-99999,AUSTRALIA,20200101,-14.7236,134.7456,16,37.7,...,26.9,5,8,12,1,0,0,1,3,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839333,95985,-99999,SWAN_ISLAND,-99999,AUSTRALIA,20231103,-40.7292,148.1250,15,17.2,...,7.1,19,21,22,20,11,11,10,10,2023-11-03
839334,95986,-99999,TASMAN_ISLAND,-99999,AUSTRALIA,20231103,-43.2397,148.0025,240,10.4,...,3.4,17,13,13,12,8,8,5,11,2023-11-03
839335,95988,-99999,MARIA_ISLAND_(POINT_LESUEUR),-99999,AUSTRALIA,20231103,-42.6619,148.0181,28,12.9,...,5.2,22,25,16,15,12,10,6,15,2023-11-03
839336,95989,-99999,BICHENO_(COUNCIL_DEPOT),-99999,AUSTRALIA,20231103,-41.8739,148.3036,11,-99999.0,...,11.1,-99999,-99999,-99999,-99999,-99999,-99999,8,-99999,2023-11-03


In [20]:
from shapely.geometry import Point

In [21]:
df_to_merged['coord'] = df_to_merged.apply(lambda x: Point(x.lon,x.lat), axis=1)
df_to_merged.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,wspd_3,wspd_6,wspd_9,wspd_12,wspd_15,wspd_18,wspd_21,wspd_24,date_new,coord
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,6,2,9,7,6,3,0,5,2020-01-01,POINT (126.6453 -14.3)
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,6,7,12,9,7,7,10,12,2020-01-01,POINT (126.15 -13.7542)
2,94103,-99999,BROWSE_ISL_AWS(AUT),-99999,AUSTRALIA,20200101,-14.1200,123.5475,5,32.4,...,19,7,4,12,13,11,11,10,2020-01-01,POINT (123.5475 -14.12)
3,94105,-99999,NOONAMAH_AIRSTRIP,-99999,AUSTRALIA,20200101,-12.7000,131.0475,18,32.8,...,2,6,3,0,1,3,3,2,2020-01-01,POINT (131.0475 -12.7)
4,94106,-99999,NGUKURR_AIRPORT,-99999,AUSTRALIA,20200101,-14.7236,134.7456,16,37.7,...,5,8,12,1,0,0,1,3,2020-01-01,POINT (134.7456 -14.7236)


In [22]:
gdf = gpd.GeoDataFrame(df_to_merged, crs="EPSG:4326", geometry=[x for x in df_to_merged.coord])
gdf.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,wspd_6,wspd_9,wspd_12,wspd_15,wspd_18,wspd_21,wspd_24,date_new,coord,geometry
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,2,9,7,6,3,0,5,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000)
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,7,12,9,7,7,10,12,2020-01-01,POINT (126.15 -13.7542),POINT (126.15000 -13.75420)
2,94103,-99999,BROWSE_ISL_AWS(AUT),-99999,AUSTRALIA,20200101,-14.1200,123.5475,5,32.4,...,7,4,12,13,11,11,10,2020-01-01,POINT (123.5475 -14.12),POINT (123.54750 -14.12000)
3,94105,-99999,NOONAMAH_AIRSTRIP,-99999,AUSTRALIA,20200101,-12.7000,131.0475,18,32.8,...,6,3,0,1,3,3,2,2020-01-01,POINT (131.0475 -12.7),POINT (131.04750 -12.70000)
4,94106,-99999,NGUKURR_AIRPORT,-99999,AUSTRALIA,20200101,-14.7236,134.7456,16,37.7,...,8,12,1,0,0,1,3,2020-01-01,POINT (134.7456 -14.7236),POINT (134.74560 -14.72360)


### Loading AUS Statistical Analysis Publicly available

In [23]:
aus_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"
local_path = './dataset/'
def data_fetching_url(url, local_path):
    r = requests.get(url)
    z = ZipFile(io.BytesIO(r.content))
#     z.extractall(path=local_path) # extract to folder
    filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
    dbf, prj, shp, shx = [filename for filename in filenames]
    gdf = gpd.read_file(local_path + shp)
    gdf_new = gdf[~gdf.isna()]
    gdf_new = gdf_new.to_crs("EPSG:4326")
    return gdf_new
aus = data_fetching_url(aus_url, local_path)

In [24]:
aus

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


#### Combining actual data with SA2 AUS

In [25]:
gdf_new = gdf.sjoin(aus[['SA2_CODE21', 'SA2_NAME21', 'STE_NAME21', 'SA3_NAME21', 'SA4_CODE21', 'geometry']])
gdf_new.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,wspd_24,date_new,coord,geometry,index_right,SA2_CODE21,SA2_NAME21,STE_NAME21,SA3_NAME21,SA4_CODE21
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,5,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,510011265,Kununurra,Western Australia,Kimberley,510
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,12,2020-01-01,POINT (126.15 -13.7542),POINT (126.15000 -13.75420),2129,510011265,Kununurra,Western Australia,Kimberley,510
42,94211,-99999,MOUNT_ELIZABETH,-99999,AUSTRALIA,20200101,-16.4200,126.1025,547,-99999.0,...,-99999,2020-01-01,POINT (126.1025 -16.42),POINT (126.10250 -16.42000),2129,510011265,Kununurra,Western Australia,Kimberley,510
45,94216,YPKU,KUNUNURRA_AIRPORT,-99999,AUSTRALIA,20200101,-15.7814,128.7100,45,37.8,...,5,2020-01-01,POINT (128.71 -15.7814),POINT (128.71000 -15.78140),2129,510011265,Kununurra,Western Australia,Kimberley,510
46,94217,-99999,ARGYLE_AERODROME,-99999,AUSTRALIA,20200101,-16.6333,128.4500,165,37.7,...,3,2020-01-01,POINT (128.45 -16.6333),POINT (128.45000 -16.63330),2129,510011265,Kununurra,Western Australia,Kimberley,510


In [26]:
gdf_new['SA2_CODE21'] = gpd_2['SA2_CODE21'].astype(str)

In [27]:
gpd_2['SA2_CODE21'] = gpd_2['SA2_CODE21'].astype(str)

In [28]:
gdf_new_new = gdf_new.drop('date_new', axis=1)
gdf_new_new

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,wspd_21,wspd_24,coord,geometry,index_right,SA2_CODE21,SA2_NAME21,STE_NAME21,SA3_NAME21,SA4_CODE21
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3000,126.6453,24,37.5,...,0,5,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510
1,94102,-99999,TROUGHTON_ISLAND,-99999,AUSTRALIA,20200101,-13.7542,126.1500,8,33.7,...,10,12,POINT (126.15 -13.7542),POINT (126.15000 -13.75420),2129,110011187,Kununurra,Western Australia,Kimberley,510
42,94211,-99999,MOUNT_ELIZABETH,-99999,AUSTRALIA,20200101,-16.4200,126.1025,547,-99999.0,...,-99999,-99999,POINT (126.1025 -16.42),POINT (126.10250 -16.42000),2129,108021156,Kununurra,Western Australia,Kimberley,510
45,94216,YPKU,KUNUNURRA_AIRPORT,-99999,AUSTRALIA,20200101,-15.7814,128.7100,45,37.8,...,2,5,POINT (128.71 -15.7814),POINT (128.71000 -15.78140),2129,108021156,Kununurra,Western Australia,Kimberley,510
46,94217,-99999,ARGYLE_AERODROME,-99999,AUSTRALIA,20200101,-16.6333,128.4500,165,37.7,...,7,3,POINT (128.45 -16.6333),POINT (128.45000 -16.63330),2129,108021156,Kununurra,Western Australia,Kimberley,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836513,94740,-99999,GUNNEDAH_RESOURCE_CENTRE,-99999,AUSTRALIA,20231030,-31.0261,150.2687,307,31.7,...,0,-99999,POINT (150.2687 -31.0261),POINT (150.26870 -31.02610),207,NaN,Gunnedah Surrounds,New South Wales,Tamworth - Gunnedah,110
837132,94740,-99999,GUNNEDAH_RESOURCE_CENTRE,-99999,AUSTRALIA,20231031,-31.0261,150.2687,307,30.7,...,0,-99999,POINT (150.2687 -31.0261),POINT (150.26870 -31.02610),207,NaN,Gunnedah Surrounds,New South Wales,Tamworth - Gunnedah,110
837747,94740,-99999,GUNNEDAH_RESOURCE_CENTRE,-99999,AUSTRALIA,20231101,-31.0261,150.2687,307,27.5,...,0,-99999,POINT (150.2687 -31.0261),POINT (150.26870 -31.02610),207,NaN,Gunnedah Surrounds,New South Wales,Tamworth - Gunnedah,110
838363,94740,-99999,GUNNEDAH_RESOURCE_CENTRE,-99999,AUSTRALIA,20231102,-31.0261,150.2687,307,28.3,...,0,-99999,POINT (150.2687 -31.0261),POINT (150.26870 -31.02610),207,NaN,Gunnedah Surrounds,New South Wales,Tamworth - Gunnedah,110


In [29]:
gdf_new.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 792622 entries, 0 to 838975
Data columns (total 44 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   stn_id       792622 non-null  int64   
 1   stn_call     792622 non-null  object  
 2   city         792622 non-null  object  
 3   state        792622 non-null  int64   
 4   country      792622 non-null  object  
 5   date         792622 non-null  object  
 6   lat          792622 non-null  float64 
 7   lon          792622 non-null  float64 
 8   elev         792622 non-null  int64   
 9   tmax         792622 non-null  float64 
 10  tmin         792622 non-null  float64 
 11  report_p     792622 non-null  float64 
 12  final_p      792622 non-null  float64 
 13  p_flag       792622 non-null  int64   
 14  num_6hr_p    792622 non-null  int64   
 15  wxchars      792622 non-null  object  
 16  trace        792622 non-null  int64   
 17  vp           792622 non-null  float64 
 18  v

In [30]:
gpd_2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   FireName     1502 non-null   object  
 1   FireNo       1502 non-null   object  
 2   Label        1502 non-null   object  
 3   StartDate    1502 non-null   object  
 4   EndDate      1502 non-null   object  
 5   AreaHa       1502 non-null   float64 
 6   PerimeterM   1502 non-null   float64 
 7   VerDate      1502 non-null   object  
 8   geometry     1502 non-null   geometry
 9   Class_Label  1502 non-null   object  
 10  SA2_CODE21   1502 non-null   object  
 11  Year         1502 non-null   int64   
dtypes: float64(2), geometry(1), int64(1), object(8)
memory usage: 140.9+ KB


In [31]:
gdf_merged = gdf_new.merge(gpd_2[['SA2_CODE21', 'Class_Label']], on="SA2_CODE21", how='left')

In [32]:
gdf_merged.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,date_new,coord,geometry,index_right,SA2_CODE21,SA2_NAME21,STE_NAME21,SA3_NAME21,SA4_CODE21,Class_Label
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
1,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
2,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
3,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
4,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,2020-01-01,POINT (126.6453 -14.3),POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn


In [33]:
gdf_merged = gdf_merged.drop(['coord', 'date_new'], axis=1)

In [34]:
gdf_merged.head()

,stn_id,stn_call,city,state,country,date,lat,lon,elev,tmax,...,wspd_21,wspd_24,geometry,index_right,SA2_CODE21,SA2_NAME21,STE_NAME21,SA3_NAME21,SA4_CODE21,Class_Label
0,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,0,5,POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
1,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,0,5,POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
2,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,0,5,POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
3,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,0,5,POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn
4,94100,-99999,KALUMBURU,-99999,AUSTRALIA,20200101,-14.3,126.6453,24,37.5,...,0,5,POINT (126.64530 -14.30000),2129,110011187,Kununurra,Western Australia,Kimberley,510,Prescribed Burn


In [ ]:
gdf_merged.to_file("2020-2023_filtered_labels_bushfire_sa2_sa3_4names")

## Testing Environment

In [ ]:
unique_dates = gdf_new.date_new.unique()
gdfs = []
unused_date = []
for date in unique_dates:
    temp = gpd_2.loc[gpd_2.StartDate==date][['SA2_CODE21', 'Class_Label']]
    if len(temp) < 1:
        unused_date.append(date)
        continue
    temp = gdf_new[gdf_new.date_new==date].merge(temp, how="left", on='SA2_CODE21')
    gdfs.append(temp)

In [ ]:
gdfs[0].head()

In [ ]:
len(unused_date) == len(unique_dates)

In [ ]:
comb_pd = pd.concat(gdfs)
comb_pd.head()

In [ ]:
comb_pd.state.unique()

In [ ]:
unique_dates = gdf.date_new.unique()

In [ ]:
gdfs = []
unused_date = []
for date in unique_dates:
    
    if len(gpd_2[gpd_2.StartDate==date]) < 1:
        unused_date.append(date)
        continue
    temp = gdf[gdf.date_new==date].sjoin(gpd_2[gpd_2.StartDate==date], how="left")
    gdfs.append(temp)

In [ ]:
gdfs[0].head()

In [ ]:
gpd_2.to_file("label_data")

In [ ]:
test_2 = gpd_2[gpd_2.StartDate == unique_dates[5]][['Class_Label', 'geometry']]
test = gdf[gdf.date_new==unique_dates[5]]

In [ ]:
test_3 = gpd.sjoin(test, test_2, how="inner", op='intersects')
test_3

In [ ]:
test_2

In [ ]:
gdf[gdf.date_new==unique_dates[0]].sjoin(gpd_2[gpd_2.StartDate == unique_dates[0]][['Class_Label', 'geometry']], how="left")

In [ ]:
gdfs = []
unused_date = []

for date in unique_dates:
    new_temp = gpd_2[gpd_2.StartDate == unique_dates[0]][['Class_Label', 'geometry']]
    if len(new_temp) < 1:
        unused_date.append(date)
        continue
    temp = gdf[gdf.date_new==unique_dates[0]].sjoin(new_temp, how="left")
    gdfs.append(temp)

In [ ]:
gdfs[5].tail()

In [ ]:
comb_df = pd.concat(gdfs)
comb_df.Class_Label.unique()

In [ ]:
comb_df.info()

In [ ]:
comb_df